# Verify skymaps and compare metrics

## Read raw skymap files

In [ ]:
from pathlib import Path
from skymap_convert import load_pickle_skymap

In [ ]:
package_root = Path.home() / "skymap-convert"

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
raw_skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

lsst_skymap = load_pickle_skymap(raw_skymap_path)
lsst_skymap

## Full Vertex skymaps

### Read skymaps

In [ ]:
from skymap_convert.test_utils import polys_are_equiv, get_poly_from_tract_id

In [ ]:
full_vertex_skymaps_dir = package_root / "converted_skymaps" / "full_vertex"

fv_inner_poly_path = full_vertex_skymaps_dir / "inner_polygons.yaml"
fv_outer_poly_path = full_vertex_skymaps_dir / "outer_polygons.yaml"

### Verify skymaps

In [ ]:
def verify_fv_polygons(fv_skymap_reader, orig_skymap, inner=True):
    for tract_id in range(orig_skymap._numTracts):
        if tract_id == 0 or tract_id == orig_skymap._numTracts - 1:
            # Skip first and last tract for now, but todo
            continue

        fv_poly = fv_skymap_reader.get_tract(tract_id).to_convex_polygon()
        orig_poly = get_poly_from_tract_id(lsst_skymap, tract_id, inner=inner)

        if not polys_are_equiv(fv_poly, orig_poly):
            raise ValueError(f"Tract {tract_id} polygons are not equivalent")
    print(f"All {'inner' if inner else 'outer'} tract polygons are equivalent.")

In [ ]:
from skymap_convert import FullVertexReader

fv_inner_reader = FullVertexReader(fv_inner_poly_path)
fv_outer_reader = FullVertexReader(fv_outer_poly_path)

In [ ]:
print("Verifying inner tract polygons...")
verify_fv_polygons(fv_inner_reader, lsst_skymap, inner=True)

print("Verifying outer tract polygons...")
verify_fv_polygons(fv_outer_reader, lsst_skymap, inner=False)

### Storage metrics

In [ ]:
print("Inner polygons file size:", round(fv_inner_poly_path.stat().st_size / (1024 * 1024), 2), "MB")
print("Outer polygons file size:", round(fv_outer_poly_path.stat().st_size / (1024 * 1024), 2), "MB")

## Ring Optimized skymaps

### Read skymaps

In [ ]:
ring_optimized_skymaps_dir = package_root / "converted_skymaps" / "ring_optimized"

ro_inner_poly_path = ring_optimized_skymaps_dir / "inner_polygons.yaml"
ro_outer_poly_path = ring_optimized_skymaps_dir / "outer_polygons.yaml"

### Verify skymaps

In [ ]:
import numpy as np
from skymap_convert.geometry import unit_vector3d_to_radec


def verify_ro_polygons(ro_skymap_reader, orig_skymap, inner=True):
    for tract_id in range(orig_skymap._numTracts):
        ro_verts = ro_skymap_reader.get_tract(tract_id).quad

        orig_poly = get_poly_from_tract_id(lsst_skymap, tract_id, inner=inner)
        orig_verts_unit_vectors = orig_poly.getVertices()
        orig_verts_radec = [unit_vector3d_to_radec(vec) for vec in orig_verts_unit_vectors]

        if not np.allclose(ro_verts, orig_verts_radec, atol=1e-6):
            raise ValueError(f"Tract {tract_id} polygons are not equivalent")

    print(f"All {'inner' if inner else 'outer'} tract polygons are equivalent.")

In [ ]:
from skymap_convert import RingOptimizedReader

ro_inner_reader = RingOptimizedReader(ro_inner_poly_path)

In [ ]:
print("Verifying inner tract polygons...")
verify_ro_polygons(ro_inner_reader, lsst_skymap, inner=True)

# No outer polygon ring optimized skymap yet

### Storage metrics

In [ ]:
print("Inner polygons file size:", round(ro_inner_poly_path.stat().st_size / 1024, 2), "KB")